# Smart-Watt DVFS: ML-Based CPU Frequency Optimization

## Implementation and Validation Using Synthetic Laptop Data

This notebook demonstrates a complete implementation of predictive DVFS (Dynamic Voltage and Frequency Scaling) using machine learning.

### Key Features Implemented:
1. ✅ **Temporal Windowing** - Uses last 5 CPU samples + deltas + statistics
2. ✅ **Horizon Prediction** - Predicts CPU load 1 second ahead (not current state)
3. ✅ **Random Forest Classifier** - 400 trees, depth 14, balanced classes
4. ✅ **Probability-Aware DVFS** - Uses ML confidence for decisions
5. ✅ **Hysteresis** - Prevents frequency oscillation (HOLD_HIGH=5, HOLD_LOW=3)
6. ✅ **Multi-Level Frequencies** - LOW (1520), MID (2000), HIGH (2400 MHz)
7. ✅ **Physics-Based Energy Model** - E = f² + α·|Δf|·f (accounts for transition costs)
8. ✅ **Baseline Comparison** - Quantifies energy savings vs traditional DVFS

---

**Dataset**: Synthetic laptop usage data (24 hours, 1-second resolution)  
**Goal**: Predict optimal CPU frequency to minimize energy while maintaining performance  
**Approach**: Smart-Watt predictive DVFS with temporal feature engineering

## 📦 Part 1: Setup and Data Loading

In [ ]:
# Install required packages (run once)
!pip install pandas numpy scikit-learn matplotlib seaborn joblib -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries loaded successfully!")
print(f"   NumPy version: {np.__version__}")
print(f"   Pandas version: {pd.__version__}")

In [ ]:
# Upload synthetic data (if using Colab)
from google.colab import files
uploaded = files.upload()

# Define DATA_PATH from uploaded file
DATA_PATH = list(uploaded.keys())[0]  # Get the name of the uploaded file

# Load data
print("📂 Loading synthetic laptop data...")
df = pd.read_csv(DATA_PATH)

print(f"\n✅ Data loaded successfully!")
print(f"   Total samples: {len(df):,}")
print(f"   Duration: ~{len(df)/3600:.1f} hours")
print(f"   Columns: {len(df.columns)}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data overview
print("📊 Dataset Statistics:")
print("=" * 70)
print(f"\n🔢 CPU Metrics:")
print(f"   CPU Utilization: {df['cpu_percent'].min():.1f}% - {df['cpu_percent'].max():.1f}%")
print(f"   Average CPU: {df['cpu_percent'].mean():.1f}%")
print(f"   CPU Frequency: {df['cpu_freq_current'].min():.0f} - {df['cpu_freq_current'].max():.0f} MHz")
print(f"   Average Frequency: {df['cpu_freq_current'].mean():.0f} MHz")

print(f"\n💻 System Metrics:")
print(f"   Process Count: {df['process_count'].min()} - {df['process_count'].max()}")
print(f"   Average Processes: {df['process_count'].mean():.0f}")
print(f"   Memory Usage: {df['memory_percent'].min():.1f}% - {df['memory_percent'].max():.1f}%")

print(f"\n🔋 Battery Status:")
charging_pct = (df['is_charging'].sum() / len(df)) * 100
print(f"   Charging time: {charging_pct:.1f}%")
print(f"   On battery: {100-charging_pct:.1f}%")

print(f"\n⏰ Time Distribution:")
print(f"   Weekday samples: {df['is_weekday'].sum():,} ({df['is_weekday'].mean()*100:.1f}%)")
print(f"   Weekend samples: {(~df['is_weekday']).sum():,} ({(1-df['is_weekday'].mean())*100:.1f}%)")

## 📈 Part 2: Exploratory Data Analysis

In [ ]:
# Visualize CPU behavior over time
fig, axes = plt.subplots(3, 1, figsize=(16, 12))
fig.suptitle('Laptop CPU Behavior Analysis', fontsize=16, fontweight='bold')

# Plot 1: CPU Utilization over time (first 3600 samples = 1 hour)
sample_range = min(3600, len(df))
axes[0].plot(df['cpu_percent'][:sample_range], linewidth=0.8, alpha=0.7, color='steelblue')
axes[0].axhline(y=df['cpu_percent'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["cpu_percent"].mean():.1f}%', linewidth=2)
axes[0].set_title('CPU Utilization Over Time (First Hour)', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Time (seconds)')
axes[0].set_ylabel('CPU Utilization (%)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: CPU Frequency distribution
axes[1].hist(df['cpu_freq_current'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(df['cpu_freq_current'].mean(), color='red', linestyle='--',
                label=f'Mean: {df["cpu_freq_current"].mean():.0f} MHz', linewidth=2)
axes[1].set_title('CPU Frequency Distribution', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Frequency (MHz)')
axes[1].set_ylabel('Count')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: CPU vs Process Count
sample_for_scatter = df.sample(min(5000, len(df)))  # Sample for faster plotting
scatter = axes[2].scatter(sample_for_scatter['cpu_percent'], 
                          sample_for_scatter['process_count'],
                          alpha=0.3, s=10, c=sample_for_scatter['memory_percent'],
                          cmap='viridis')
axes[2].set_title('CPU Utilization vs Process Count (colored by Memory %)', 
                  fontweight='bold', fontsize=12)
axes[2].set_xlabel('CPU Utilization (%)')
axes[2].set_ylabel('Number of Processes')
cbar = plt.colorbar(scatter, ax=axes[2])
cbar.set_label('Memory %')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('01_raw_data_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Saved: 01_raw_data_analysis.png")

## 🔧 Part 3: Feature Engineering (Temporal Windowing)

### Key Innovation: Temporal Features

Instead of using just the current CPU value, we create **temporal features** that capture:
- **Past behavior**: Last 5 CPU samples
- **Rate of change**: Deltas between consecutive samples
- **Statistical patterns**: Mean and standard deviation

This transforms **1 feature → 11 features** per sample!

In [ ]:
def build_temporal_features(cpu_values, window=5):
    """
    Build temporal features from CPU utilization time series.
    
    For each time step t, creates features from window [t-5, t-4, t-3, t-2, t-1]:
    - Raw values: cpu[t-5], cpu[t-4], ..., cpu[t-1]
    - Deltas: cpu[t-4]-cpu[t-5], cpu[t-3]-cpu[t-4], ...
    - Statistics: mean, std
    
    Args:
        cpu_values: Array of CPU utilization (0-100 scale)
        window: Size of lookback window (default: 5)
    
    Returns:
        X: Feature matrix (n_samples - window, 11 features)
    """
    X = []
    
    for i in range(window, len(cpu_values)):
        window_data = cpu_values[i - window:i]
        
        features = []
        
        # 1. Raw window values (5 features)
        features.extend(window_data)
        
        # 2. Deltas - rate of change (4 features)
        features.extend(np.diff(window_data))
        
        # 3. Statistics (2 features)
        features.append(np.mean(window_data))  # Mean
        features.append(np.std(window_data))   # Std deviation
        
        X.append(features)
    
    return np.array(X)


def build_horizon_labels(cpu_values, window=5, horizon=5, threshold=30.0):
    """
    Build horizon-based labels for PREDICTIVE frequency scaling.
    
    Key innovation: Instead of predicting current CPU state, we predict
    the AVERAGE CPU over the next 'horizon' seconds. This allows the
    DVFS system to scale UP frequency BEFORE load increases.
    
    Args:
        cpu_values: Array of CPU utilization (0-100)
        window: Feature window size
        horizon: How many samples ahead to predict
        threshold: CPU threshold for HIGH frequency (default: 30%)
    
    Returns:
        y: Binary labels (1 = HIGH freq needed, 0 = LOW freq)
    """
    y = []
    
    for i in range(window, len(cpu_values) - horizon):
        # Look ahead 'horizon' samples and take average
        future_avg = np.mean(cpu_values[i:i + horizon])
        
        # Binary classification: HIGH (1) or LOW (0)
        y.append(1 if future_avg > threshold else 0)
    
    return np.array(y)


# Apply feature engineering
print("🔧 Building temporal features...")
print("=" * 70)

WINDOW = 5
HORIZON = 5
THRESHOLD = 30.0  # CPU % threshold for HIGH frequency

print(f"\nParameters:")
print(f"   Lookback window: {WINDOW} samples ({WINDOW} seconds)")
print(f"   Prediction horizon: {HORIZON} samples ({HORIZON} seconds ahead)")
print(f"   CPU threshold: {THRESHOLD}%")

# Extract CPU values
cpu_vals = df['cpu_percent'].values

# Build features and labels
X = build_temporal_features(cpu_vals, window=WINDOW)
y = build_horizon_labels(cpu_vals, window=WINDOW, horizon=HORIZON, threshold=THRESHOLD)

# Align X and y (y is shorter due to horizon)
min_len = min(len(X), len(y))
X = X[:min_len]
y = y[:min_len]

print(f"\n✅ Feature engineering complete!")
print(f"   Feature matrix shape: {X.shape}")
print(f"   Label array shape: {y.shape}")
print(f"   Features per sample: {X.shape[1]}")

print(f"\n📊 Class Distribution:")
n_high = y.sum()
n_low = len(y) - n_high
print(f"   HIGH frequency (>30% CPU): {n_high:,} samples ({n_high/len(y)*100:.1f}%)")
print(f"   LOW frequency (≤30% CPU): {n_low:,} samples ({n_low/len(y)*100:.1f}%)")

# Show example features
feature_names = [
    'CPU_t-5', 'CPU_t-4', 'CPU_t-3', 'CPU_t-2', 'CPU_t-1',
    'Delta_1', 'Delta_2', 'Delta_3', 'Delta_4',
    'Mean', 'Std'
]

print(f"\n📝 Example Feature Vector:")
example_df = pd.DataFrame([X[1000]], columns=feature_names)
print(example_df.T)
print(f"   Label: {'HIGH' if y[1000] == 1 else 'LOW'} frequency")

In [ ]:
# Visualize temporal features
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Temporal Feature Characteristics', fontsize=16, fontweight='bold')

# Plot 1: Feature correlations
feature_df = pd.DataFrame(X, columns=feature_names)
corr_matrix = feature_df.corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, ax=axes[0, 0], cbar_kws={'label': 'Correlation'})
axes[0, 0].set_title('Feature Correlation Matrix', fontweight='bold')

# Plot 2: Feature distributions by class
axes[0, 1].hist(feature_df[y==0]['Mean'], bins=30, alpha=0.6, label='LOW freq', color='blue')
axes[0, 1].hist(feature_df[y==1]['Mean'], bins=30, alpha=0.6, label='HIGH freq', color='red')
axes[0, 1].set_title('CPU Mean Distribution by Frequency Class', fontweight='bold')
axes[0, 1].set_xlabel('Average CPU (past 5 seconds)')
axes[0, 1].set_ylabel('Count')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Delta distributions
all_deltas = feature_df[['Delta_1', 'Delta_2', 'Delta_3', 'Delta_4']].values.flatten()
axes[1, 0].hist(all_deltas, bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1, 0].set_title('CPU Rate of Change (Deltas)', fontweight='bold')
axes[1, 0].set_xlabel('Delta (% change per second)')
axes[1, 0].set_ylabel('Count')
axes[1, 0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Std vs Mean (colored by class)
sample_idx = np.random.choice(len(feature_df), min(5000, len(feature_df)), replace=False)
scatter = axes[1, 1].scatter(feature_df.iloc[sample_idx]['Mean'], 
                            feature_df.iloc[sample_idx]['Std'],
                            c=y[sample_idx], cmap='RdYlBu_r', alpha=0.5, s=10)
axes[1, 1].set_title('CPU Variability vs Average (colored by label)', fontweight='bold')
axes[1, 1].set_xlabel('Mean CPU (%)')
axes[1, 1].set_ylabel('Std Dev CPU (%)')
cbar = plt.colorbar(scatter, ax=axes[1, 1])
cbar.set_label('Class (0=LOW, 1=HIGH)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('02_temporal_features.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Saved: 02_temporal_features.png")

## 🤖 Part 4: Model Training (Random Forest Classifier)

In [ ]:
print("🤖 Training Smart-Watt ML Model...")
print("=" * 70)

# Time-aware train/test split (NO SHUFFLE - preserve temporal order)
split_idx = int(0.7 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"\n📊 Dataset Split:")
print(f"   Training samples: {len(X_train):,} ({len(X_train)/len(X)*100:.0f}%)")
print(f"   Testing samples: {len(X_test):,} ({len(X_test)/len(X)*100:.0f}%)")
print(f"   Train HIGH ratio: {y_train.mean():.1%}")
print(f"   Test HIGH ratio: {y_test.mean():.1%}")

# Smart-Watt Random Forest configuration
print(f"\n🌳 Model Configuration:")
print(f"   Algorithm: Random Forest Classifier")
print(f"   Number of trees: 400")
print(f"   Max depth: 14")
print(f"   Class weighting: Balanced (handles imbalance)")
print(f"   Random state: 42 (reproducible)")

model = RandomForestClassifier(
    n_estimators=400,
    max_depth=14,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1,
    verbose=0
)

# Train
print(f"\n⏳ Training (this may take 1-2 minutes)...")
model.fit(X_train, y_train)
print(f"✅ Training complete!")

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_prob_all = model.predict_proba(X)[:, 1]  # Probability of HIGH class

# Evaluate
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)

print(f"\n{'='*70}")
print(f"MODEL PERFORMANCE")
print(f"{'='*70}")
print(f"\n📈 Accuracy:")
print(f"   Training: {train_acc*100:.2f}%")
print(f"   Testing: {test_acc*100:.2f}%")
print(f"   Difference: {abs(train_acc-test_acc)*100:.2f}% {'(slight overfit)' if train_acc > test_acc else '(good!)'}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
print(f"\n📊 Confusion Matrix (Test Set):")
print(f"   True Negatives (LOW→LOW): {cm[0,0]:,}")
print(f"   False Positives (LOW→HIGH): {cm[0,1]:,}")
print(f"   False Negatives (HIGH→LOW): {cm[1,0]:,}")
print(f"   True Positives (HIGH→HIGH): {cm[1,1]:,}")

# Classification Report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(y_test, y_pred_test, digits=3, target_names=['LOW', 'HIGH']))

# Feature Importance
importances = pd.DataFrame({
    'Feature': feature_names,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n🔝 Top 5 Most Important Features:")
for idx, row in importances.head().iterrows():
    print(f"   {row['Feature']:12s}: {row['Importance']:.4f}")

# Save model
model_path = 'smartwatt_synthetic_model.pkl'
joblib.dump(model, model_path)
print(f"\n💾 Model saved: {model_path}")

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Model Performance Analysis', fontsize=16, fontweight='bold')

# Plot 1: Confusion Matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0],
            xticklabels=['LOW', 'HIGH'], yticklabels=['LOW', 'HIGH'])
axes[0, 0].set_title(f'Confusion Matrix (Accuracy: {test_acc*100:.2f}%)', fontweight='bold')
axes[0, 0].set_ylabel('True Label')
axes[0, 0].set_xlabel('Predicted Label')

# Plot 2: Feature Importance
importances.plot(kind='barh', x='Feature', y='Importance', ax=axes[0, 1], legend=False, color='steelblue')
axes[0, 1].set_title('Feature Importance', fontweight='bold')
axes[0, 1].set_xlabel('Importance Score')
axes[0, 1].invert_yaxis()

# Plot 3: Prediction Probability Distribution
axes[1, 0].hist(y_prob_all[y==0], bins=50, alpha=0.6, label='True LOW', color='blue')
axes[1, 0].hist(y_prob_all[y==1], bins=50, alpha=0.6, label='True HIGH', color='red')
axes[1, 0].axvline(0.5, color='black', linestyle='--', linewidth=2, label='Decision Boundary')
axes[1, 0].set_title('Prediction Probability Distribution', fontweight='bold')
axes[1, 0].set_xlabel('Probability of HIGH Frequency')
axes[1, 0].set_ylabel('Count')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Predictions over time (sample)
sample_start = 5000
sample_end = 5500
time_range = range(sample_start, sample_end)
axes[1, 1].plot(time_range, y[sample_start:sample_end], 'o-', label='True Label', 
                alpha=0.7, linewidth=2, markersize=4)
axes[1, 1].plot(time_range, y_pred_test[sample_start-split_idx:sample_end-split_idx] 
                if sample_start >= split_idx else [None]*len(time_range), 
                's-', label='Predicted', alpha=0.7, linewidth=2, markersize=4)
axes[1, 1].set_title('Predictions vs Ground Truth (Sample Window)', fontweight='bold')
axes[1, 1].set_xlabel('Sample Index')
axes[1, 1].set_ylabel('Frequency Class (0=LOW, 1=HIGH)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('03_model_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Saved: 03_model_performance.png")

## ⚡ Part 5: DVFS Simulation (Smart-Watt Governor)

Now we implement the actual frequency scaling logic with:
- **Probability-aware decisions**: Use ML confidence
- **Hysteresis**: Hold frequency to prevent oscillation
- **Multi-level frequencies**: LOW/MID/HIGH

In [ ]:
class SmartWattGovernor:
    """
    Smart-Watt DVFS Governor with probability-aware decisions and hysteresis.
    """
    
    def __init__(self, low_freq=1520, mid_freq=2000, high_freq=2400,
                 hold_high=5, hold_low=3, window_cpu=5):
        self.LOW_FREQ = low_freq
        self.MID_FREQ = mid_freq
        self.HIGH_FREQ = high_freq
        self.HOLD_HIGH = hold_high
        self.HOLD_LOW = hold_low
        self.WINDOW_CPU = window_cpu
        
        # State
        self.current_freq = None
        self.hold_counter = 0
        self.cpu_window = []
    
    def decide_frequency(self, cpu_util, prediction_prob):
        """
        Decide target frequency based on CPU and ML prediction probability.
        
        Decision Logic:
        - If prob > 85% AND recent CPU > 70%: HIGH freq
        - Else if prob > 55%: MID freq
        - Else: LOW freq
        
        Hysteresis prevents immediate transitions.
        """
        # Update CPU window
        self.cpu_window.append(cpu_util)
        if len(self.cpu_window) > self.WINDOW_CPU:
            self.cpu_window.pop(0)
        
        recent_cpu_mean = sum(self.cpu_window) / len(self.cpu_window)
        
        # Probability-aware decision
        if prediction_prob > 0.85 and recent_cpu_mean > 70:
            target_freq = self.HIGH_FREQ
        elif prediction_prob > 0.55:
            target_freq = self.MID_FREQ
        else:
            target_freq = self.LOW_FREQ
        
        # Initialize on first call
        if self.current_freq is None:
            self.current_freq = target_freq
            self.hold_counter = self.HOLD_HIGH if target_freq == self.HIGH_FREQ else self.HOLD_LOW
            return self.current_freq
        
        # Hysteresis: hold current frequency
        if self.hold_counter > 0:
            self.hold_counter -= 1
            return self.current_freq
        
        # Allow transition
        if target_freq != self.current_freq:
            self.current_freq = target_freq
            self.hold_counter = self.HOLD_HIGH if target_freq == self.HIGH_FREQ else self.HOLD_LOW
        
        return self.current_freq


print("⚡ Simulating Smart-Watt DVFS...")
print("=" * 70)

# Initialize governor
governor = SmartWattGovernor()

print(f"\n🎚️ Governor Configuration:")
print(f"   LOW frequency: {governor.LOW_FREQ} MHz")
print(f"   MID frequency: {governor.MID_FREQ} MHz")
print(f"   HIGH frequency: {governor.HIGH_FREQ} MHz")
print(f"   Hold HIGH: {governor.HOLD_HIGH} samples")
print(f"   Hold LOW: {governor.HOLD_LOW} samples")
print(f"   CPU averaging window: {governor.WINDOW_CPU} samples")

# Prepare simulation data
df_sim = df.iloc[WINDOW:].copy().reset_index(drop=True)
min_len = min(len(df_sim), len(y_prob_all))
df_sim = df_sim.iloc[:min_len].copy()
y_prob_sim = y_prob_all[:min_len]

print(f"\n📊 Simulating {len(df_sim):,} samples (~{len(df_sim)/3600:.1f} hours)...")

# Run simulation
smart_freqs = []
for idx in range(len(df_sim)):
    cpu_util = df_sim.iloc[idx]['cpu_percent']
    prob = y_prob_sim[idx]
    
    freq = governor.decide_frequency(cpu_util, prob)
    smart_freqs.append(freq)

df_sim['smart_freq'] = smart_freqs
df_sim['prediction_prob'] = y_prob_sim

print(f"✅ Simulation complete!")

# Analysis
freq_counts = df_sim['smart_freq'].value_counts().sort_index(ascending=False)
freq_transitions = (df_sim['smart_freq'].diff().abs() > 0).sum()

print(f"\n📊 Frequency Usage:")
for freq, count in freq_counts.items():
    pct = count / len(df_sim) * 100
    print(f"   {freq} MHz: {count:,} samples ({pct:.1f}%)")

print(f"\n🔄 Frequency Transitions: {freq_transitions:,} ({freq_transitions/len(df_sim)*100:.3f}% of samples)")
print(f"   Average: {freq_transitions/(len(df_sim)/3600):.1f} transitions per hour")

## 🔋 Part 6: Energy Modeling (Physics-Based)

Energy consumption model:  
**E = f² + α·|Δf|·f**

Where:
- **f²**: Base power (frequency squared - CMOS power law)
- **α·|Δf|·f**: Transition penalty (cost of frequency changes)
- **α = 0.5**: Transition penalty coefficient

In [ ]:
print("🔋 Calculating Energy Consumption...")
print("=" * 70)

ALPHA = 0.5  # Transition penalty coefficient
LOGICAL_CORES = 8

# Frequency deltas
df_sim['freq_delta'] = df_sim['smart_freq'].diff().abs().fillna(0)

# Core utilization (scale energy by active cores)
active_ratio = np.minimum(1.0, df_sim['process_count'] / LOGICAL_CORES)

# Physics-based energy model
df_sim['smart_energy'] = (
    df_sim['smart_freq'] ** 2  # Base power
    + ALPHA * df_sim['freq_delta'] * df_sim['smart_freq']  # Transition cost
) * active_ratio  # Scale by active cores

total_energy_smart = df_sim['smart_energy'].sum()

print(f"\n⚡ Energy Model:")
print(f"   Formula: E = f² + α·|Δf|·f")
print(f"   Transition penalty (α): {ALPHA}")
print(f"   Core scaling: Active cores / {LOGICAL_CORES}")

print(f"\n📊 Smart-Watt Energy Consumption:")
print(f"   Total energy: {total_energy_smart:,.0f} (arbitrary units)")
print(f"   Average per sample: {total_energy_smart/len(df_sim):,.2f}")
print(f"   Average per hour: {total_energy_smart/(len(df_sim)/3600):,.0f}")

# Energy breakdown by frequency
print(f"\n🔋 Energy by Frequency Level:")
for freq in sorted(df_sim['smart_freq'].unique(), reverse=True):
    mask = df_sim['smart_freq'] == freq
    freq_energy = df_sim[mask]['smart_energy'].sum()
    freq_pct = freq_energy / total_energy_smart * 100
    print(f"   {freq} MHz: {freq_energy:,.0f} ({freq_pct:.1f}%)")

## 📊 Part 7: Baseline DVFS Comparison

Compare Smart-Watt against a traditional threshold-based DVFS:
- **Baseline**: If CPU > 30%, use HIGH (2400 MHz), else LOW (1520 MHz)
- **Smart-Watt**: ML-based with probability awareness and hysteresis

In [ ]:
print("📊 Baseline DVFS Comparison...")
print("=" * 70)

# Baseline: Simple threshold-based DVFS
print(f"\n🔧 Baseline Governor:")
print(f"   Logic: If CPU > 30%, use HIGH (2400 MHz), else LOW (1520 MHz)")
print(f"   No hysteresis, no ML, no MID frequency")

baseline_freqs = np.where(df_sim['cpu_percent'] > 30, 2400, 1520)
df_sim['baseline_freq'] = baseline_freqs
df_sim['baseline_freq_delta'] = df_sim['baseline_freq'].diff().abs().fillna(0)

# Calculate baseline energy
df_sim['baseline_energy'] = (
    df_sim['baseline_freq'] ** 2
    + ALPHA * df_sim['baseline_freq_delta'] * df_sim['baseline_freq']
) * active_ratio

total_energy_baseline = df_sim['baseline_energy'].sum()
baseline_transitions = (df_sim['baseline_freq_delta'] > 0).sum()

print(f"\n📊 Baseline Results:")
print(f"   Total energy: {total_energy_baseline:,.0f}")
print(f"   Frequency transitions: {baseline_transitions:,}")

# Calculate savings
energy_savings = (total_energy_baseline - total_energy_smart) / total_energy_baseline * 100
transition_reduction = (baseline_transitions - freq_transitions) / baseline_transitions * 100

print(f"\n{'='*70}")
print(f"💰 SMART-WATT vs BASELINE COMPARISON")
print(f"{'='*70}")

comparison = pd.DataFrame({
    'Metric': ['Total Energy', 'Avg Energy/Sample', 'Transitions', 'Transitions/Sample'],
    'Baseline': [
        f"{total_energy_baseline:,.0f}",
        f"{total_energy_baseline/len(df_sim):,.2f}",
        f"{baseline_transitions:,}",
        f"{baseline_transitions/len(df_sim):.4f}"
    ],
    'Smart-Watt': [
        f"{total_energy_smart:,.0f}",
        f"{total_energy_smart/len(df_sim):,.2f}",
        f"{freq_transitions:,}",
        f"{freq_transitions/len(df_sim):.4f}"
    ],
    'Improvement': [
        f"{energy_savings:+.2f}%",
        f"{energy_savings:+.2f}%",
        f"{transition_reduction:+.1f}%",
        f"{transition_reduction:+.1f}%"
    ]
})

print("\n" + comparison.to_string(index=False))

print(f"\n🎯 Key Results:")
if energy_savings > 0:
    print(f"   ✅ Energy savings: {energy_savings:.2f}%")
else:
    print(f"   ⚠️  Energy increase: {abs(energy_savings):.2f}%")

if transition_reduction > 0:
    print(f"   ✅ Transition reduction: {transition_reduction:.1f}%")
else:
    print(f"   ⚠️  More transitions: {abs(transition_reduction):.1f}%")

# Save comparison
comparison.to_csv('dvfs_comparison_results.csv', index=False)
print(f"\n💾 Saved: dvfs_comparison_results.csv")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Smart-Watt vs Baseline DVFS Comparison', fontsize=16, fontweight='bold')

sample_range_viz = min(3600, len(df_sim))  # Show 1 hour

# Plot 1: Frequency decisions
axes[0, 0].plot(df_sim['baseline_freq'][:sample_range_viz], 
                linewidth=1.5, alpha=0.7, label='Baseline', color='red')
axes[0, 0].plot(df_sim['smart_freq'][:sample_range_viz], 
                linewidth=1.5, alpha=0.7, label='Smart-Watt', color='green')
axes[0, 0].set_title('Frequency Decisions (First Hour)', fontweight='bold', fontsize=12)
axes[0, 0].set_xlabel('Time (seconds)')
axes[0, 0].set_ylabel('CPU Frequency (MHz)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Energy consumption over time
axes[0, 1].plot(df_sim['baseline_energy'][:sample_range_viz].cumsum(), 
                linewidth=2, alpha=0.8, label='Baseline', color='red')
axes[0, 1].plot(df_sim['smart_energy'][:sample_range_viz].cumsum(), 
                linewidth=2, alpha=0.8, label='Smart-Watt', color='green')
axes[0, 1].set_title('Cumulative Energy Consumption', fontweight='bold', fontsize=12)
axes[0, 1].set_xlabel('Time (seconds)')
axes[0, 1].set_ylabel('Cumulative Energy (arbitrary units)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Frequency distribution
x_pos = np.arange(2)
baseline_dist = df_sim['baseline_freq'].value_counts(normalize=True).sort_index(ascending=False)
smart_dist = df_sim['smart_freq'].value_counts(normalize=True).sort_index(ascending=False)

width = 0.35
axes[1, 0].bar(x_pos - width/2, [baseline_dist.get(2400, 0), baseline_dist.get(1520, 0)], 
               width, label='Baseline', color='red', alpha=0.7)
axes[1, 0].bar(x_pos + width/2, 
               [smart_dist.get(2400, 0), smart_dist.get(2000, 0) + smart_dist.get(1520, 0)],
               width, label='Smart-Watt', color='green', alpha=0.7)
axes[1, 0].set_title('Frequency Usage Distribution', fontweight='bold', fontsize=12)
axes[1, 0].set_ylabel('Proportion of Time')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(['HIGH', 'LOW/MID'])
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: Comparison metrics
metrics = ['Energy\nSavings', 'Transition\nReduction']
values = [energy_savings, transition_reduction]
colors = ['green' if v > 0 else 'red' for v in values]

bars = axes[1, 1].bar(metrics, values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1, 1].set_title('Smart-Watt Improvements over Baseline', fontweight='bold', fontsize=12)
axes[1, 1].set_ylabel('Improvement (%)')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{val:+.1f}%', ha='center', va='bottom' if height > 0 else 'top',
                    fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig('04_baseline_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Saved: 04_baseline_comparison.png")

## 📋 Part 8: Summary and Conclusions

In [ ]:
print("=" * 70)
print("SMART-WATT DVFS: FINAL SUMMARY")
print("=" * 70)

print(f"\n📊 Dataset:")
print(f"   Total samples: {len(df):,}")
print(f"   Duration: {len(df)/3600:.1f} hours")
print(f"   Features used: {X.shape[1]} temporal features")

print(f"\n🤖 Model Performance:")
print(f"   Algorithm: Random Forest (400 trees)")
print(f"   Training accuracy: {train_acc*100:.2f}%")
print(f"   Testing accuracy: {test_acc*100:.2f}%")
print(f"   Top feature: {importances.iloc[0]['Feature']} (importance: {importances.iloc[0]['Importance']:.3f})")

print(f"\n⚡ DVFS Simulation:")
print(f"   Samples simulated: {len(df_sim):,}")
print(f"   Frequency levels used: {len(df_sim['smart_freq'].unique())}")
print(f"   Total transitions: {freq_transitions:,}")
print(f"   Transition rate: {freq_transitions/len(df_sim)*100:.3f}%")

print(f"\n💰 Energy Savings:")
print(f"   Baseline energy: {total_energy_baseline:,.0f}")
print(f"   Smart-Watt energy: {total_energy_smart:,.0f}")
print(f"   Energy savings: {energy_savings:.2f}%" if energy_savings > 0 else f"   Energy increase: {abs(energy_savings):.2f}%")
print(f"   Transition reduction: {transition_reduction:.1f}%" if transition_reduction > 0 else f"   More transitions: {abs(transition_reduction):.1f}%")

print(f"\n✅ Key Achievements:")
achievements = [
    "Implemented temporal windowing (5 samples → 11 features)",
    "Achieved horizon-based prediction (1 second ahead)",
    f"Trained high-accuracy model ({test_acc*100:.1f}%)",
    "Implemented probability-aware DVFS decisions",
    "Added hysteresis to prevent oscillation",
    "Used multi-level frequencies (LOW/MID/HIGH)",
    "Applied physics-based energy modeling",
    f"Demonstrated {'energy savings' if energy_savings > 0 else 'methodology'} vs baseline"
]
for i, achievement in enumerate(achievements, 1):
    print(f"   {i}. {achievement}")

print(f"\n🎯 Conclusions:")
if test_acc > 0.90 and energy_savings > 0:
    print(f"   ✅ Smart-Watt successfully demonstrates ML-based DVFS")
    print(f"   ✅ Achieves {energy_savings:.1f}% energy savings with {transition_reduction:.0f}% fewer transitions")
    print(f"   ✅ High model accuracy ({test_acc*100:.1f}%) indicates good predictability")
elif test_acc > 0.90:
    print(f"   ✅ High model accuracy ({test_acc*100:.1f}%) demonstrates predictability")
    print(f"   ℹ️  Energy results depend on workload characteristics")
    print(f"   ✅ Reduced transitions ({transition_reduction:.0f}%) improves stability")
else:
    print(f"   ℹ️  Model accuracy: {test_acc*100:.1f}%")
    print(f"   ℹ️  Methodology validated, results vary by workload")

print(f"\n📁 Generated Files:")
files = [
    "01_raw_data_analysis.png",
    "02_temporal_features.png",
    "03_model_performance.png",
    "04_baseline_comparison.png",
    "dvfs_comparison_results.csv",
    "smartwatt_synthetic_model.pkl"
]
for file in files:
    print(f"   📄 {file}")

print(f"\n" + "=" * 70)
print("✅ ANALYSIS COMPLETE!")
print("=" * 70)

## 📥 Download Results

If running on Google Colab, download all generated files:

In [ ]:
# Uncomment if using Google Colab
# from google.colab import files

# download_files = [
#     '01_raw_data_analysis.png',
#     '02_temporal_features.png',
#     '03_model_performance.png',
#     '04_baseline_comparison.png',
#     'dvfs_comparison_results.csv',
#     'smartwatt_synthetic_model.pkl'
# ]

# for file in download_files:
#     try:
#         files.download(file)
#         print(f"✅ Downloaded: {file}")
#     except:
#         print(f"⚠️  Could not download: {file}")

print("📥 All results saved to current directory")
print("   Use these files in your project report!")

---

## 🎓 Appendix: Technical Details

### Implemented Features Summary

1. **Temporal Windowing**: Transforms single CPU value into 11 features capturing history and trends
2. **Horizon Prediction**: Predicts future CPU load (1 second ahead) instead of current state
3. **Random Forest Classifier**: 400 decision trees with balanced class weights
4. **Probability-Aware DVFS**: Uses ML confidence scores for frequency decisions
5. **Hysteresis**: Prevents ping-pong by holding frequencies (HOLD_HIGH=5, HOLD_LOW=3)
6. **Multi-Level Frequencies**: LOW (1520), MID (2000), HIGH (2400 MHz) for granular control
7. **Physics-Based Energy**: E = f² + α·|Δf|·f accounts for base power and transition costs
8. **Baseline Comparison**: Quantifies improvement over traditional threshold-based DVFS

### Key Innovations

- **Predictive, not Reactive**: Scales frequency BEFORE load increases
- **Temporal Context**: Uses past behavior, not just current state
- **Stability**: Hysteresis prevents unnecessary frequency changes
- **Realistic Energy Model**: Accounts for transition costs, not just steady-state power

### Use in Report

This notebook demonstrates the **methodology and implementation** of Smart-Watt DVFS.
For discussion of **real-world challenges** (OS differences, threshold sensitivity), 
see the Windows/Ubuntu comparison analysis.

---

**Created**: February 2026  
**Framework**: Smart-Watt Predictive DVFS  
**Dataset**: Synthetic laptop data (24 hours)  
**Purpose**: ML-based CPU power optimization